#Activity 05 - Experiments

***
##### CS 434 - Dating Mining and Machine Learning
##### Oregon State University-Cascades
***

# Load packages

In [3]:
import ssl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import MinMaxScaler
ssl._create_default_https_context = ssl._create_unverified_context

# Dataset

[Dataset information](http://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset)

Bike sharing systems are new generation of traditional bike rentals where whole process from membership, rental and return back has become automatic. Through these systems, user is able to easily rent a bike from a particular position and return back at another position. Currently, there are about over 500 bike-sharing programs around the world which is composed of over 500 thousands bicycles. Today, there exists great interest in these systems due to their important role in traffic, environmental and health issues.

This dataset contains the hourly and daily count of rental bikes between years 2011 and 2012 in Capital bikeshare system with the corresponding weather and seasonal information.

### Features

* **instant** : record index
* **dteday** : date
* **season** : season (1:winter, 2:spring, 3:summer, 4:fall)
* **yr** : year (0:2011, 1:2012)
* **mnth** : month (1 to 12)
* **hr** : hour (0 to 23)
* **holiday** : whether day is holiday or not
* **weekday** : day of the week
* **workingday** : if day is neither weekend nor holiday is 1, otherwise is 0.
* **weathersit** :
  1. Clear, Few clouds, Partly cloudy, Partly cloudy
  2. Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
  3. Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
  4. Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
* **temp** : Normalized temperature in Celsius. The values are derived via * $(t-t_{min})/(t_{max}-t_{min})$, $t_{min}=-8$, $t_{max}=+39$ (only in hourly scale)
* **atemp**: Normalized feeling temperature in Celsius. The values are derived via $(t-t_{min})/(t_{max}-t_{min})$, $t_{min}=-16$, $t_{max}=+50$ (only in hourly scale)
* **hum**: Normalized humidity. The values are divided to 100 ($max$)
* **windspeed**: Normalized wind speed. The values are divided to 67 ($max$)
* **casual**: count of casual users
* **registered**: count of registered users

### Outcome
* **cnt**: count of total rental bikes including both casual and registered

*** 
# Exercise #1 - Load data
*** 

1.1 Download data file and `unzip`.

In [24]:
# run this code
!wget 'http://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip'
!unzip 'Bike-Sharing-Dataset.zip'
# you only need to run once
# you may comment these lines out after running (in case you run-all later)

--2020-04-14 21:33:22--  http://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)...128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80...connected.
HTTP request sent, awaiting response...200 OK
Length: 279992 (273K) [application/x-httpd-php]
Saving to: ‘Bike-Sharing-Dataset.zip.1’

Bike-Sharing-Datase 100%[===================>] 273.43K  1.05MB/s    in 0.3s    

2020-04-14 21:33:22 (1.05 MB/s) - ‘Bike-Sharing-Dataset.zip.1’ saved [279992/279992]

Archive:  Bike-Sharing-Dataset.zip
replace Readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:

1.2 Load the dataset from the url and `display(df)`.

> Header is included in the `.csv`

In [10]:
# load the dataset into a dataframe
file = 'day.csv'
df = clean_df = pd.read_csv(file)
display(df)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,2012-12-27,1,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,247,1867,2114
727,728,2012-12-28,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,644,2451,3095
728,729,2012-12-29,1,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,159,1182,1341
729,730,2012-12-30,1,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,364,1432,1796


In [19]:
# drop 'instant' and 'dteday'
# clean_df.drop(columns=['instant', 'dteday'], inplace=True)

display(clean_df['season'])

0      1
1      1
2      1
3      1
4      1
      ..
726    1
727    1
728    1
729    1
730    1
Name: season, Length: 731, dtype: int64

In [9]:
# count the number of rows
print(df.shape[0])

731


In [14]:
##### 1.5 Describe the data

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,2.496580,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,1.110807,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,3.000000,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000
max,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


In [18]:
# describe the data
display(clean_df.describe())

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,2.496580,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,1.110807,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,3.000000,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000
max,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


*** 
# Exercise #2 - Build a pipeline
*** 

##### 2.1 Split features and class output

In [26]:
# split to X and y
X, y = clean_df.loc[:, clean_df.columns != 'season'].values, clean_df.loc[:, clean_df.columns == 'season'].values


##### 2.2 Partition to train and test (0.33)

In [27]:
# partition to train and test
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, 
                     test_size=0.33,
                     stratify=y,
                     random_state=1)

##### 2.4 Build a pipeline

> For your pipeline, use:
* `StandardScaler()`
* `RandomForestRegressor(n_estimators=10, max_features='sqrt')` for the classifier.  

Do not use `PCA`.

In [28]:
# build a pipeline
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression(random_state=1, solver='lbfgs'))

pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
print('Test Accuracy: %.3f' % pipe_lr.score(X_test, y_test))

Test Accuracy: 0.591
/home/gravebry/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


##### 2.5 Train model and predict test set

> Save your prediction in variable `y_pred`.

In [10]:
# train your pipeline; predict test set; display accuracy
display(type(y_pred))

your code here


##### Self Check

In [11]:
# predict train set and print accuracy
print('Train Accuracy: %.3f' % pipe_lr.score(X_train, y_train))

NameError: name 'pipe_lr' is not defined

> `Train Accuracy: 0.990`

*** 
# Exercise #3 - Explore the results
***

Now, let's examine these results to get a sense of of our predictions.

##### 3.1 Print `y_test` and see the `y` true values

In [12]:
# print y_test
print('your code here')

your code here


##### 3.2 Print `y_pred` and see the predicted `y` values

In [13]:
# print `y_pred`
print('your code here')

your code here


##### 3.3 Make a new dataframe with true `y` values and predicted `y` values

Combine your true and predicted values into one new dataframe
* use `pd.DataFrame(..)` to construct the dataframe
* name the new dataframe `df_result`
* it will have two columns
* add `y_test` as column `'true'`
* add `y_pred` as column `'pred'`

> **Example**: create a dataframe
>
>```df_new = pd.DataFrame({'A':[1,2,3], 'B':[4,5,6], 
                   'C':[7,8,9], 'D':[10,11,12]})```

In [14]:
# create new dataframe df_results with 'true' (ytest) and 'pred' (y_pred)
print('your code here')

your code here


##### 3.4 Create a new column `'error'` in `df_results`

The error for any predicted example is:
$$ error_i = \ \mid true_i - pred_i \ \mid $$

In [15]:
# create new column 'error' in df_results
print('your code here')

your code here


##### 3.5 Normalize the `'error'` column and display `df_results`

In [16]:
# min-max scale the error to [0..1]
print('your code here')

your code here


> Note: we normalized our error $[0,1]$ for use in visualization in the next section. 

##### 3.6 Describe `df_results`

In [17]:
# describe df_results
print('your code here')

your code here


*** 
# Exercise #4 - Visualizing results
*** 

> A graph without title and axes labels is often no more useful than no graph at all. Always set a title and label your axes.

##### 4.1 Graph `'true'` vs `'pred'` test

Scatterplot your `df_results`:
* $x$-axis is `'true'`
* $y$-axis is `'pred'`
* `xlabel` is 'True'
* `label` is 'Predicted'
* `title` is 'Preciction Results'

In [18]:
# scatter plot 'true' vs 'pred'
print('your code here')

your code here


> Here we should see the `'true'` values along x-axis and the `'predicted'` values along the y-axis. You should see the points clustered in an "almost" line along the diagnonal. 

##### 4.2 Add color to better visualize the error

Scatter plot your `df_results` again
* repeat your code from `4.1`
* modify to add color `c` equal to the normalized `'error'` values

In [19]:
# graph 'true' vs 'pred'
print('your code here')

your code here


4.3 Add regressive line to graph

Scatter plot your `df_results` yet again
* repeat your code from `4.2`
* add line `m, b = np.polyfit(x, y, 1)`
* add line `plt.plot(x, m*x + b, color='red')`
* where `x` and `y` are your `'true'` and `'pred'` series

In [20]:
# graph 'true' vs 'pred' with line
print('your code here')

your code here


> Now that's a *perdy* plot.

4.4 Visualize the learning curve

Copy block of code from `Lecture 05` in order to plot the learning curve. Modify to use the pipeline (and classifier) you used in this `Activity 05`. Set your `ylim` to be $[0.75, 1.03]$.

In [21]:
# plot the learning
print('your code here')

your code here


> **Self Check**: 
>
> The first <font color="blue">blue</font> point is $\approx (48,0.97)$. 
>
> The first <font color="green">green</font> point varies in `y` (try re-running!) in the  range of  $\approx$ $(48,0.77)$ to $\approx$ $(48,0.82)$.

*** 
# Exercise #5 - $k$-fold cross-validation
*** 

Earlier we split the data into $1\over3$ `test` and $2\over3$ `train`. While this is fine practice for very large datasets, for small datasets we run the risk of producing a non-representive `test` set.  Let's investigate this possibility by examinig results with cross-validation.

5.1 Perform cross-fold validation.

Use [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) cross-validation with $k=10$.

> This is a regression problem and our predictor is continuous. You don't want `StratifiedKFold` for this reason. 

In [22]:
# Use KFold cross-fold validation with k=10
print('your code here')

your code here


> Cross-validation is the proper approach for all but very large datasets.

<img src="https://66.media.tumblr.com/dded9d1a2bf2068f92af9f7a9b6b5451/tumblr_p6s3hbPzgV1vd8jsjo1_500.gifv" width="300">